<a href="https://colab.research.google.com/github/hadagarcia/llm-zoomcamp/blob/main/workshop/LLM_zoomcamp_RAG_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop RAG using dlt and LanceDB - Homework

# Create an up-to-date RAG with dlt and LanceDB

We will be creating an LLM chat bot that has the latest knowledge of the employee handbook of a fictional company. We will be able to chat to it about specific policies like PTO, work from home etc.

To build this, we would need to do three things:
1. The company policies exist in a [Notion Page](https://dlthub.notion.site/Employee-handbook-669c2a1e04044465811c8ca22977685d). We will need to first extract the text from these pages.
2. Once extracted, we will want to embed them into vectors and then store them in a vector database.
3. This will allow us to create our RAG: a function that would accept a user question, match it to the information stored in the vector database, and then send the question + relevant information as input to the LLM.

We will be using the following OSS tools for this:
1. dlt for data ingestion:  
  1. dlt can easily connect to any REST API source (like Notion)
  2. It also has integrations with vector databases, like LanceDB.
  3. It also allows to easily plug in functionality like incremental loading.
2. LanceDB as a vector database:
  1. LanceDB is an open-source vector database that is very easy to use and integrate into python workflows
  2. It is in-process and serverless (like DuckDB), which makes querying and retreival very efficient
3. Ollama for RAG:
  1. Ollama is open-source and allows you to easily run LLMs locally

**Note on running this notebook**: We are going to download and use a local Ollama instance for the RAG, so preferably select the **T4 GPU** in the runtime when starting this notebook (Runtime > Change runtime type > Hardware accelerator > T4 GPU).

You can also use the default CPU in case you're facing technical issues, but then your LLM responses might be slower (~2 mins/response)

## Part 1: Create a Notion -> LanceDB pipeline using dlt

### 1. Install requirements

To create a notion -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [1]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

### 2. Create a dlt project with rest_api source and lancedb destination

We now create a dlt project using the command `dlt init <source> <destination>`.

This downloads all the modules required for the dlt source (rest api, in this case) into the local directory. See the side panel for the directory structure created.

What is the dlt rest api source?

It is a dlt source that allows you to connect to any REST API endpoint using a declarative configuration. You can:
- pass the endpoints that you want to connect to,
- define the relation between the endpoints
- define how you want to handle pagination and authentication

In [2]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


### 3. Add API credentials

To access APIs, databases, or any third-party applications, one might need to specify relevant credentials.



In [3]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

### 4. Write the pipeline code

**Note**: We first go over the code step by step before putting it into runnable cells

1. Import necessary modules (run this cell)

In [4]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

### 5. Run the pipeline

In [5]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework", #Here we specify to query onle the pages that contain the string homework [hidden page = "Homework: Employee handbook"]
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content, # It contains all the content for each page respectively
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    last_value = last_edited_time.last_value
    print(f'Last value [last_edited_time]: {last_value}')

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content" # We need to embed only the content of the page
        ),
        table_name="homework", #employee_handbook
        write_disposition="merge"
    )
    print(load_info)

load_notion()

Last value [last_edited_time]: 2024-06-26T08:16:00.000Z
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_version
[{'name': 'version', 'data_type':

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

UPLOAD
Pipeline company_policies load step completed in 31.24 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7b5cf5c71cc0> location to store data
Load package 1721771029.4618268 is LOADED and contains no failed jobs


### 6. Visualize the output

In [6]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework") # previously it was notion_pages___employee_handbook

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265623, 0.04746083, -0.011796438, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-23 21:43:50.682988+00:00,1721771029.4618268,hJTem1sFfQ4SRQ
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.049661573, 0.10853516, -0.0097625945, -0.0...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-23 21:43:50.687368+00:00,1721771029.4618268,vD9YIslEBfbOuA
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316319, 0.17331506, 0.025351718, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.687639+00:00,1721771029.4618268,eT0CEK3OapkfgA
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974315, 0.10586075, 0.0032906067, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.687843+00:00,1721771029.4618268,l28sW4cYCHuRAQ
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423306, -0.06457594, 0.065863, 0.0145438...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.688034+00:00,1721771029.4618268,29CHCG6aiMq3lQ
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.0005233448, -0.054883398, 0.043573365, -0.0...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.688224+00:00,1721771029.4618268,Hp9d1AchVqnsgA
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.03802632, -0.021509668, 0.0475278, 0.064706...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.688414+00:00,1721771029.4618268,ztIDhydvdAsSIA
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.058588073, -0.07540443, 0.033775173, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.688622+00:00,1721771029.4618268,OHh8MwzPLBBsGA
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.01359926, 0.004753031, 0.024835143, 0.0159...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-23 21:43:50.688846+00:00,1721771029.4618268,iRX1G7T8QAix2w
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.032060914, 0.02424462, 0.008471355, 0.03179...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-23 21:43:50.689025+00:00,1721771029.4618268,/K/q4yoXE90U0g


In [14]:
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___homework']


## Q1: Rows in LanceDB
How many rows does the lancedb table "notion_pages__homework" have?

In [7]:
len(dbtable)

17

 ---

## Q2: Running the Pipeline: Last edited time

In the demo, we created an incremental dlt resource rest_api_notion_incremental that keeps track of last_edited_time. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column last_edited_time of the table)

In [8]:
# Convert to pandas DataFrame and get the max last_edited_time
import pandas as pd
df = dbtable.to_pandas()

### What value does it store after you've run your pipeline once?

In [9]:
df['last_edited_time'] = pd.to_datetime(df['last_edited_time'], utc=True)
max_last_edited_time = df['last_edited_time'].max()

# Format the timestamp
timestamp_format1 = max_last_edited_time.strftime("Timestamp('%Y-%m-%d %H:%M:%S+0000', tz='UTC')")
timestamp_format2 = max_last_edited_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")

print(f'The most recent last_edited_time is: {timestamp_format1} (OR "{timestamp_format2})")')

The most recent last_edited_time is: Timestamp('2024-07-05 23:33:00+0000', tz='UTC') (OR "2024-07-05T23:33:00.000Z)")


## Part 2: Create a RAG bot using Ollama

With the contents from the employee handbook vectorized and stored in LanceDB, we're now ready to create our RAG with Ollama.


1. Install Ollama into the notebook's local runtime

In [10]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


2. Start Ollama using `ollama serve`. This needs to run in the backgound - so we run it using `nohup` (to see the output log, open nohup.out).

In [20]:
!nohup ollama serve > nohup.out 2>&1 &

3. Pull the desired model. We're going to be using `llama1-uncensored` (takes about 1m to download)

In [21]:
%%capture
!ollama pull llama2-uncensored

In this next part we're going to be writing functions that accept user question, retrieve the relevant paragraphs from lancedb, and the pass the question and the retrieved pages as input into the ollama chat assistant

4. pip install ollama and import it

In [13]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [15]:
import ollama

5. Write a function that can retrieve content from lancedb relevant to the user query
  
  With LanceDB, you don't have to explicity embed the question. LanceDB stores information on the embedding model used and automatically embeds the question.

  We use the `db_table.search()` function to query the DB and then limit it to the top 2 most similar results and return that as the context to pass to the RAG.

  Limiting results is important because otherwise there might be too much confusing information. Similarly only picking the top choice might not give enough information.

In [16]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

6. Finally we define a very basic RAG. We define a simple system prompt, retrieve the relevant context for the user query with the function defined above and then send the user question and the context to the `llama2-uncensored` model.

In [17]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

And we run the RAG! Some example questions you can ask:

* How many vacation days do I get?
* Can I get maternity leave?

**Note**: This is a very basic implementation of a RAG, since this workshop is mainly about data ingestion. So expect some weird answers. If you do stop and restart the cell, you will need to rerun the cell containing `ollama serve` first.

In [22]:
main()

You: how many PTO days are the employees entitled to in a year?
Assistant: "Thank you for asking about the employee handbook. Based on what I read, it looks like employees at this company are eligible for 30 days of paid time off (PTO) per year. That means if you have worked with us for a year, you will receive 25 PTO days in total. This includes any holidays that the company offers as well.
If you need clarification on something specific or want more information about your employee benefits, please feel free to reach out to our HR department for assistance."
You: I got confused, is it 30 or 25 PTO days in total per year?
Assistant: "Thank you for asking about the employee handbook. Based on what I read, it looks like employees at this company are eligible for 30 days of paid time off (PTO) per year. That means if you have worked with us for a year, you will receive 25 PTO days in total. This includes any holidays that the company offers as well.
However, I noticed that you seemed conf

KeyboardInterrupt: Interrupted by user

There's a lot more to learn and do with dlt and LanceDB, find more info the [dlt docs](https://dlthub.com/docs/) and the [LanceDB docs](https://lancedb.github.io/lancedb/)

If you have questions about this workshop or dlt, feel free to join our [community on Slack](https://dlthub.com/community).

If you're at EuroPython in Prague this week, come see us at our booth!